In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [3]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [4]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

c:\Users\MSI\AppData\Local\pypoetry\Cache\virtualenvs\bank-customer-dQCuybCs-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### Chunking

In [6]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("C:/rag/data/youth-offerings.pdf"),
    PyPDFLoader("C:/rag/data/insurance-products-and-services.pdf"),
    PyPDFLoader("C:/rag/data/loan-offerings.pdf"),
    PyPDFLoader("C:/rag/data/savings-accounts.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
len(docs)

36

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)
len(splits)

42

In [8]:
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=hf,
    persist_directory=persist_directory
)

In [9]:
question = "do you have online app?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [10]:
print(docs)

[Document(metadata={'page': 4, 'source': 'C:/rag/data/youth-offerings.pdf'}, page_content='Youth Offerings | 5Absa Ignition Account\nSpark your journey to financial greatness\nBegin the journey towards the life you want to live with an \naccount that is as youthful and ambitious as you are.\nIf you are between the ages of 18 and 24, this account \nis for you. It allows you to transact anywhere, anytime. It \nwill give you transactional convenience as well as financial \nconnectivity and this is the spark that you need to get \nthings done.  \nWhat you should know\n• You will need P50 to open this account\n• You must maintain a minimum operating balance of \nP10\n• You will get free SMS alerts everytime you transact \n• This account gives you the convenience on banking \n24/7 via your mobile and online\n• We don’t charge you for eStatements\n• You are guaranteed an interest of up to 1.5% if you \nmaintain a  minimum of P50 in your account \n• We will only charge you an affordable monthl

In [12]:
print(type(llm))

<class 'langchain_huggingface.llms.huggingface_endpoint.HuggingFaceEndpoint'>


### Generation

In [13]:
from langchain.prompts import PromptTemplate
template = """You're Inetbank assistant, always Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "What can i do if forget my password of my mobile banking?"
qa_chain = RetrievalQA.from_chain_type(chat_model,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

C:\Users\MSI\AppData\Local\Temp\ipykernel_10716\104539818.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


'If you forget your password for mobile banking, you can follow these steps to retrieve it:\n\n1. Go to the Absa Mobile Banking login page.\n\n2. Click on "Forgot Password" or "Reset Password" option.\n\n3. Enter your username or customer number and your Secure Registration Code (SRC) or your ATM card number and PIN.\n\n4. Follow the instructions provided to recover your password. This may involve'